In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
from nltk.text import sent_tokenize
import os
import numpy as np
import pandas as pd
import sys
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import json
import operator
import time
import math
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter,defaultdict
from natsort import natsorted
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
Input: chuỗi kí tự documents_path là đường dẫn tới thư mục chứa các tài liệu
Output: danh sách documents, trong đó mỗi phần tử của danh sách là toàn bộ chuỗi kí tự của tập tài liệu tương ứng
Ví dụ: documents = ['the specific configuration . the experiment . ',
                    'the discussion here is restricted . two dimensional incompressible steady flow .']
'''
def read_documents(documents_path):
    documents = []
    os.chdir(documents_path)
    # Use natsorted() for natural sorting of file names
    for file in natsorted(os.listdir()):
    # Check whether file is in text format or not
        if file.endswith(".txt"):
            file_path = f"{documents_path}/{file}"
            with open(file_path, 'r') as f:
                documents.append(f.read())
    return documents

#ĐỌC DỮ LIỆU

In [ ]:
documents_path = '/content/drive/MyDrive/CS419.Q11-UIT/Cranfield/Cranfield'
documents = read_documents(documents_path)
print(documents[:5])

['experimental investigation of the aerodynamics of a wing in a slipstream . an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem . the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary layer control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory . an empirical evaluation of the destalling effects was made for the specific configuration of the experiment . ', "simple shear flow past a flat plate in an incompressible fluid of small viscosity . in th

#ĐỌC FILE QUERY

In [ ]:
'''
Input: chuỗi kí tự queries_path là đường dẫn tới tập tin chứa các câu truy vấn
Output: danh sách queries, trong đó mỗi phần tử của danh sách là toàn bộ chuỗi kí tự của câu truy vấn tương ứng
Ví dụ: queries = ['aeroelastic models of heated high speed aircraft .',
                  'with flight of high speed aircraft .']
'''
def read_queries(queries_path):
    with open(queries_path) as file:
        queries = file.readlines()
    # Mỗi dòng trong tập tin query.txt có cấu trúc [id]\t[câu truy vấn]
    # -> Ta lấy câu truy vấn bằng cách tách từng dòng bởi dấu \t và chỉ lấy phần sau sau dấu \t
    for idx in range(len(queries)):
        queries[idx] = queries[idx][:-1].split("\t")[1]
    return queries

In [ ]:
queries_path = '/content/drive/MyDrive/CS419.Q11-UIT/Cranfield/TEST/query.txt'
queries = read_queries(queries_path)
print(queries[:5])

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .', 'what are the structural and aeroelastic problems associated with flight of high speed aircraft .', 'what problems of heat conduction in composite slabs have been solved so far .', 'can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium .', 'what chemical kinetic system is applicable to hypersonic aerodynamic problems .']


#ĐỌC FILE RES


In [ ]:
path_res = '/content/drive/MyDrive/CS419.Q11-UIT/Cranfield/TEST/RES/{}.txt'
RES = list()
for index in range(0,225):
  f = open(path_res.format(index+1),'r', encoding='UTF-8')
  data = f.read()
  b = data.split()
  tempt = list()
  for i in range(1,len(b),3):
    tempt.append(int(b[i]))
  RES.append(tempt)
f.close()

In [ ]:
print(RES[0])
print(len(RES))

[184, 29, 31, 12, 51, 102, 13, 14, 15, 57, 378, 859, 185, 30, 37, 52, 142, 195, 875, 56, 66, 95, 462, 497, 858, 876, 879, 880, 486]
225


#XÁC ĐỊNH TERM

In [ ]:
def update_inverted_index(id, term, inverted_index):
    if term in inverted_index:
        if id in inverted_index[term]:
            inverted_index[term][id] += 1
        else:
            inverted_index[term][id] = 1
    else:
        inverted_index[term] = {id: 1}
    return inverted_index

In [ ]:
# Hàm này dùng để phân tách term từ 1 câu
def create_terms(sentence):
    terms = []
    # Remove punctuation and numbers. Split on whitespace
    regex = r"[ .,()0123456789=:+-/']\s*"
    word_lst = re.split(regex, sentence)
    # Use filter(None, ) to remove empty element after splitting string
    word_lst = filter(None, word_lst)
    for word in word_lst:
        # Remove stopwords
        stopword_lst = stopwords.words('english')
        if word in stopword_lst:
            continue
        # Perform stemming
        stemmer = SnowballStemmer("english")
        stem_word = stemmer.stem(word)
        # Remove 1 character
        #if len(stem_word) < 3:
        #    continue
        terms.append(stem_word)
    return terms

In [ ]:
def create_inverted_index(id, sentence, inverted_index):
    terms = create_terms(sentence)
    for term in terms:
        # Update term in inverted_index
        inverted_index = update_inverted_index(id, term, inverted_index)
    return inverted_index

In [ ]:
inverted_index = {}
idx = 1
for document in documents:
    inverted_index = create_inverted_index(idx, document, inverted_index)
    idx += 1
print(inverted_index)

{'experiment': {1: 2, 11: 1, 12: 1, 16: 1, 17: 1, 19: 1, 25: 1, 29: 1, 30: 2, 35: 1, 37: 1, 41: 1, 42: 1, 43: 1, 47: 1, 52: 1, 53: 1, 58: 1, 69: 1, 70: 1, 74: 1, 78: 2, 84: 2, 99: 2, 101: 1, 103: 1, 112: 1, 115: 1, 121: 1, 123: 3, 131: 1, 137: 1, 140: 1, 142: 1, 154: 1, 156: 1, 167: 1, 168: 1, 170: 1, 171: 2, 173: 2, 176: 1, 179: 2, 183: 1, 184: 1, 186: 3, 187: 1, 188: 1, 189: 1, 191: 1, 195: 3, 197: 2, 202: 1, 203: 1, 206: 2, 207: 2, 212: 1, 216: 1, 220: 1, 222: 1, 225: 2, 227: 1, 230: 1, 234: 4, 245: 1, 251: 1, 256: 2, 257: 1, 262: 1, 271: 2, 273: 1, 277: 1, 282: 1, 283: 1, 286: 1, 287: 1, 289: 1, 294: 1, 295: 1, 304: 1, 307: 1, 329: 2, 330: 2, 334: 2, 338: 1, 339: 1, 344: 2, 345: 1, 346: 3, 347: 1, 354: 1, 360: 1, 369: 1, 370: 1, 372: 2, 377: 1, 397: 1, 409: 1, 411: 2, 413: 2, 418: 1, 420: 1, 421: 1, 423: 1, 427: 1, 433: 1, 435: 1, 439: 1, 441: 2, 442: 3, 443: 1, 453: 1, 455: 2, 462: 1, 464: 1, 467: 1, 484: 3, 494: 2, 496: 1, 497: 1, 498: 1, 501: 1, 503: 1, 504: 1, 505: 1, 511: 1, 5

#TIỀN XỬ LÝ DỮ LIỆU

In [ ]:
def text_process(content):
    stop_list = stopwords.words('english')
    stemmer = SnowballStemmer("english")

    # Xóa khoảng trắng đầu
    content = content.lstrip()

    # Chỉ giữ chữ cái
    content = re.sub('[^A-Za-z]+', ' ', content)

    # Chuyển lowercase
    content = content.lower()

    # Tokenize
    tokens = nltk.word_tokenize(content)

    # Loại stopword + stemming
    stem_tokens = [
        stemmer.stem(word)
        for word in tokens
        if word not in stop_list
    ]

    # Ghép chuỗi để lọc từ ngắn
    clean_stem_tokens = ' '.join(stem_tokens)

    # Loại từ có độ dài 1–2 ký tự
    shortword = re.compile(r'\b\w{1,2}\b')
    clean_stem_tokens = shortword.sub('', clean_stem_tokens)

    # Tokenize lại lần cuối
    return word_tokenize(clean_stem_tokens)


In [ ]:
t = text_process(documents[0])
t

['experiment',
 'investig',
 'aerodynam',
 'wing',
 'slipstream',
 'experiment',
 'studi',
 'wing',
 'propel',
 'slipstream',
 'made',
 'order',
 'determin',
 'spanwis',
 'distribut',
 'lift',
 'increas',
 'due',
 'slipstream',
 'differ',
 'angl',
 'attack',
 'wing',
 'differ',
 'free',
 'stream',
 'slipstream',
 'veloc',
 'ratio',
 'result',
 'intend',
 'part',
 'evalu',
 'basi',
 'differ',
 'theoret',
 'treatment',
 'problem',
 'compar',
 'span',
 'load',
 'curv',
 'togeth',
 'support',
 'evid',
 'show',
 'substanti',
 'part',
 'lift',
 'increment',
 'produc',
 'slipstream',
 'due',
 'destal',
 'boundari',
 'layer',
 'control',
 'effect',
 'integr',
 'remain',
 'lift',
 'increment',
 'subtract',
 'destal',
 'lift',
 'found',
 'agre',
 'well',
 'potenti',
 'flow',
 'theori',
 'empir',
 'evalu',
 'destal',
 'effect',
 'made',
 'specif',
 'configur',
 'experi']

In [ ]:
def word_dictionary(data):
  d = []
  doc = []
  for sent in range(0,len(data)):
    temp = text_process(data[sent])
    for term in temp:
      if term not in d:
        d.append(term)
    doc.append(temp)
  return d,doc

In [ ]:
word_list,doc = word_dictionary(documents)

In [ ]:
len(doc)

1400

In [ ]:
len(word_list)

4142

In [ ]:
def text_to_binary_vector(sent,word_list):
    # Create a binary vector with the same length as the word list
    binary_vector = [0] * len(word_list)
    for word in sent:
        if word in word_list:
            index = word_list.index(word)
            binary_vector[index] = 1
    return binary_vector

In [ ]:
def represent_data(data,word_list):
  doc = []
  for sent in range(0,len(data)):
    doc.append(text_to_binary_vector(data[sent],word_list))
  return doc

In [ ]:
ivan = represent_data(doc,word_list)

In [ ]:
len(ivan)

1400

In [ ]:
df1 = pd.DataFrame(ivan,index=range(1,1401),columns=word_list)

In [ ]:
df1.head(5)

,experiment,investig,aerodynam,wing,slipstream,studi,propel,made,order,determin,...,cauchi,reformul,delin,aeolotrop,determinant,intuit,hoshizaki,recover,fralich,ing
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#THỐNG KÊ MỤC TỪ

In [ ]:
def create_vocab_lst(inverted_index):
    vocab_lst = {}
    for term in inverted_index:
        freq = len(inverted_index[term])
        no_of_docs = sum(inverted_index[term].values())
        vocab_lst[term] = [freq, no_of_docs]
    return vocab_lst

In [ ]:
'''
Hàm này dùng để cập nhật thông tin của các mục từ trong từ điển word_lst
# Input:
# + id: id của câu
# + term: term của tài liệu
# + word_lst: từ điển word_lst chứa thông tin
# Output: từ điển word_lst sau khi đã khi đã cập nhật thông tin term
'''
def update_word_lst(id, word, word_lst):
    if word in word_lst:
        if id in word_lst[word]:
            # Cập nhật tần số xuất hiện của từ đó lên 1
            word_lst[word][id] += 1
        else:
            # Thêm chỉ số tài liệu và tần số = 1 tương ứng với mục từ mới
            word_lst[word][id] = 1
    else:
        # Thêm mục từ mới vào trong word_lst và chỉ số tài liệu, tần số = 1 tương ứng
        word_lst[word] = {id: 1}
    return word_lst

In [ ]:
# Hàm này dùng để phân tách mục từ từ 1 câu
def create_words(sentence):
    words = []
    # Remove punctuation and numbers. Split on whitespace
    regex = r"[ .,()0123456789=:+-/']\s*"
    words = re.split(regex, sentence)
    # Use filter(None, ) to remove empty element after splitting string
    words = filter(None, words)
    return words

In [ ]:
# Hàm này dùng để cập nhật thông tin toàn bộ mục từ của 1 câu trong từ điển word_lst
def create_word_lst(id, sentence, word_lst):
    words = create_words(sentence)
    for word in words:
        # Update word in word_lst
        word_lst = update_word_lst(id, word, word_lst)
    return word_lst

In [ ]:
word_lst = {}
idx = 1
for document in documents:
    word_lst = create_word_lst(idx, document, word_lst)
    idx += 1
print(word_lst)

{'experimental': {1: 2, 11: 1, 12: 1, 17: 1, 19: 1, 25: 1, 29: 1, 30: 1, 35: 1, 41: 1, 42: 1, 47: 1, 52: 1, 53: 1, 58: 1, 69: 1, 70: 1, 74: 1, 78: 1, 84: 2, 99: 2, 101: 1, 103: 1, 112: 1, 115: 1, 121: 1, 123: 2, 137: 1, 140: 1, 142: 1, 154: 1, 156: 1, 168: 1, 170: 1, 171: 1, 173: 2, 176: 1, 179: 2, 183: 1, 184: 1, 186: 2, 187: 1, 188: 1, 189: 1, 191: 1, 195: 3, 197: 2, 202: 1, 203: 1, 206: 2, 207: 2, 212: 1, 216: 1, 220: 1, 222: 1, 225: 1, 227: 1, 230: 1, 234: 4, 245: 1, 251: 1, 256: 2, 257: 1, 262: 1, 271: 2, 273: 1, 277: 1, 282: 1, 283: 1, 286: 1, 294: 1, 295: 1, 304: 1, 307: 1, 329: 2, 330: 1, 334: 2, 338: 1, 339: 1, 344: 2, 345: 1, 346: 2, 347: 1, 354: 1, 360: 1, 369: 1, 370: 1, 372: 2, 377: 1, 397: 1, 409: 1, 411: 2, 413: 2, 418: 1, 420: 1, 421: 1, 423: 1, 427: 1, 435: 1, 439: 1, 441: 1, 442: 2, 453: 1, 455: 2, 462: 1, 464: 1, 467: 1, 484: 3, 494: 2, 496: 1, 497: 1, 498: 1, 501: 1, 503: 1, 504: 1, 505: 1, 511: 1, 518: 1, 520: 2, 522: 3, 536: 1, 540: 1, 544: 2, 549: 1, 552: 2, 553:

In [ ]:
word_info_lst = create_vocab_lst(word_lst)
res_path = '/content/drive/MyDrive/CS419.Q11-UIT/word_lst.txt'
with open(res_path, 'w') as file:
    for word in word_info_lst:
        string = word + ' ' + str(word_info_lst[word][0]) + ' ' + str(word_info_lst[word][1]) + '\n'
        file.write(string)

In [ ]:
print('Số lượng mục từ có trong toàn bộ tập tài liệu là:', len(word_info_lst))

Số lượng mục từ có trong toàn bộ tập tài liệu là: 7052


#LẬP CHỈ MỤC

In [ ]:
def index_doc(doc,word_list):
  indexing = []
  for i in range(len(word_list)):
    sum = 0
    list_doc = []
    for d in range(len(doc)):
      if doc[d][i] >=1:
        list_doc.append(d+1)
        sum+=1
    indexing.append((word_list[i],sum,list_doc))
  return indexing

In [ ]:
idx = index_doc(ivan,word_list)

In [ ]:
idx[0][0]

'experiment'

In [ ]:
from collections import defaultdict

inverted_index = defaultdict(set)

for doc_id, text in enumerate(documents, start=1):
    tokens = text_process(text)
    for term in tokens:
        inverted_index[term].add(doc_id)


In [ ]:
vocab_lst = create_vocab_lst(inverted_index)
res_path = '/content/drive/MyDrive/CS419.Q11-UIT/vocab_lst.txt'
with open(res_path, 'w') as file:
    for term in vocab_lst:
        string = term + ' ' + str(vocab_lst[term][0]) + ' ' + str(vocab_lst[term][1]) + '\n'
        file.write(string)

In [ ]:
print('Số lượng terms có trong toàn bộ tập tài liệu là:', len(inverted_index))

Số lượng terms có trong toàn bộ tập tài liệu là: 4239


In [ ]:
res_path = '/content/drive/MyDrive/CS419.Q11-UIT/posting_lst.txt'
with open(res_path, 'w') as file:
    for term in inverted_index:
        idx = 1
        for id in inverted_index[term]:
            string = term + ' ' + str(id) + ' ' + str(inverted_index[term][id]) + '\n'
            file.write(string)

In [ ]:
for term in sorted(inverted_index):
    print(term, ":", inverted_index[term])


abbrevi : {122}
abil : {738, 51, 77}
abl : {99, 132, 581, 1114, 908, 914, 695, 986, 763}
ablat : {1096, 553, 1065, 587, 1097, 1098, 1099, 1100, 1101, 1226, 82, 274, 536, 1241, 1279}
abrupt : {576, 992, 588, 1039, 662, 439}
absenc : {1281, 966, 1321, 1323, 499, 757, 1237, 152, 1147}
absent : {628}
absolut : {1003, 460, 1035, 1261, 562, 1010, 1395, 62, 474, 987, 414}
absorb : {353, 163, 164, 1200, 881, 373, 1147, 1244, 1279}
absorpt : {1346, 1316, 357, 166, 1097, 620}
abstract : {154, 479}
abund : {718}
academ : {344}
acceler : {768, 896, 1163, 788, 662, 29, 33, 34, 290, 164, 292, 813, 814, 1201, 562, 51, 698, 832, 1346, 1219, 967, 330, 459, 208, 83, 339, 595, 1242, 620}
acceleromet : {882}
accentu : {1169}
accept : {1153, 1346, 1347, 388, 101, 841, 1002, 1261, 1370, 656, 753, 723, 819, 1242, 1019, 253, 575}
access : {1105}
accident : {715}
accommod : {1056, 518, 168, 974, 1239, 1147}
accompani : {261, 296, 520, 109, 589, 207, 724, 53, 152, 1277, 1147, 797, 638}
accomplish : {192, 355, 1

In [ ]:
def index_query(query,word_list):
  indexing = []
  for d in range(len(query)):
    list_term = []
    for i in range(len(word_list)):
      if query[d][i] >=1:
        list_term.append(word_list[i])
    indexing.append(list_term)
  return indexing

In [ ]:
def info(data):
  quantity = []
  index_doc_num = []
  words = []
  for index in range(0,len(data)):
    words.append(data[index][0])
    quantity.append(data[index][1])
    index_doc_num.append(data[index][2])
  return  words,quantity, index_doc_num

In [ ]:
#BOOLEAN

In [ ]:
def boolean_retrieval(query, inverted_index):
    retrieved_indexes = []
    query_terms = create_terms(query)
    for term in query_terms:
        if term in inverted_index:
            retrieved_indexes += inverted_index[term]
    return list(set(retrieved_indexes))

In [ ]:
print(boolean_retrieval(queries[1], inverted_index))

[1, 2, 4, 6, 7, 9, 11, 12, 13, 14, 15, 16, 19, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 44, 45, 46, 47, 49, 51, 52, 53, 58, 60, 66, 67, 69, 70, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 87, 92, 93, 95, 96, 97, 98, 99, 100, 101, 102, 103, 107, 108, 110, 114, 115, 116, 122, 124, 125, 129, 130, 132, 139, 141, 142, 143, 144, 145, 146, 149, 152, 155, 156, 157, 159, 160, 162, 163, 164, 166, 169, 170, 171, 172, 173, 174, 176, 177, 179, 181, 183, 184, 185, 186, 187, 189, 193, 195, 197, 198, 201, 202, 203, 204, 205, 206, 209, 211, 212, 213, 214, 215, 216, 217, 220, 222, 223, 225, 226, 229, 230, 235, 236, 237, 238, 239, 240, 242, 244, 245, 246, 248, 251, 252, 253, 255, 262, 263, 264, 266, 272, 274, 275, 280, 281, 283, 284, 285, 287, 288, 290, 292, 293, 296, 297, 300, 302, 304, 305, 309, 311, 315, 316, 318, 319, 320, 321, 322, 323, 328, 329, 334, 336, 342, 344, 345, 346, 347, 349, 350, 351, 352, 353, 359, 360, 362, 363, 364, 365, 366, 368, 369, 370, 373, 374, 375,

In [ ]:
'''
Input: chuỗi kí tự relevance_path là đường dẫn tới thư mục chứa kết quả truy vấn đúng
Output: từ điển relevance, trong đó:
+ thành phần khóa là id của câu truy vấn
+ thành phần value là danh sách id của tập tài liệu tương ứng với kết quả truy vấn đúng dựa trên id của câu truy vấn trên
Ví dụ: relevance = {1: [1, 2, 3]
                    2: [3, 4]}
'''
def read_relevance(relevance_path):
    relevance = {}
    os.chdir(relevance_path)
    idx = 1
    # Use natsorted() for natural sorting of file names
    for file in natsorted(os.listdir()):
    # Check whether file is in text format or not
        if file.endswith(".txt"):
            file_path = f"{relevance_path}/{file}"
            with open(file_path, 'r') as f:
                line = f.readlines()
                relevance[idx] = []
                for li in line:
                    temp = re.split(r"[ \t\n]\s*", li)
                    relevance[idx].append(int(temp[1]))
        idx += 1
    return relevance

In [ ]:
relevance_path = '/content/drive/MyDrive/CS419.Q11-UIT/Cranfield/TEST/RES'
relevance = read_relevance(relevance_path)
print(relevance)

{1: [184, 29, 31, 12, 51, 102, 13, 14, 15, 57, 378, 859, 185, 30, 37, 52, 142, 195, 875, 56, 66, 95, 462, 497, 858, 876, 879, 880, 486], 2: [12, 15, 184, 858, 51, 102, 202, 14, 52, 380, 746, 859, 948, 285, 390, 391, 442, 497, 643, 856, 857, 877, 864, 658, 486], 3: [5, 6, 90, 91, 119, 144, 181, 399, 485], 4: [236, 166, 488], 5: [552, 401, 1297, 1296, 488], 6: [99, 115, 257, 258, 491], 7: [20, 56, 57, 58, 19, 492], 8: [48, 122, 20, 58, 196, 354, 360, 197, 999, 1112, 1005, 492], 9: [21, 22, 550, 534], 10: [259, 405, 302, 436, 437, 438, 998, 1011, 493], 11: [27, 28, 262, 160, 20, 263, 654, 495], 12: [86, 194, 650, 649, 652, 624], 13: [64, 265, 65, 311, 496], 14: [64, 65, 496], 15: [463, 462, 497], 16: [266, 106, 196, 498], 17: [106, 196, 498], 18: [196, 197, 198, 498], 19: [32, 67, 164, 639, 715, 716, 719, 1379, 717, 499], 20: [87, 88, 104, 267, 268, 269, 270, 407, 408, 500], 21: [271, 16, 413, 414, 502], 22: [68, 502], 23: [900, 902, 200, 201, 601, 899, 903, 593, 199, 594, 901, 544, 597, 

In [ ]:
# Dùng for duyệt một số câu truy vấn đầu. So sánh đối chiếu với
# kết quả trong relevance và đưa ra nhận xét
res = []
precision_lst = []
recall_lst = []
f_score_lst = []
no_queries = len(queries)
for idx in range(0, no_queries):
    retrieved_indexes = set(boolean_retrieval(queries[idx], inverted_index))
    relevant_indexes = set(relevance[idx+1])
    no_relevant_docs_retrieved = len(retrieved_indexes & relevant_indexes)

    precision, recall, f_score = 0, 0, 0

    if len(retrieved_indexes) != 0:
        precision = round(no_relevant_docs_retrieved/len(retrieved_indexes), 2)
    if len(relevant_indexes) != 0:
        recall = round(no_relevant_docs_retrieved/len(relevant_indexes), 2)
    if precision * recall != 0:
        f_score = round((2 * precision * recall) / (precision + recall), 2)

    precision_lst.append(precision)
    recall_lst.append(recall)
    f_score_lst.append(f_score)

In [ ]:
res_path = '/content/drive/MyDrive/CS419.Q11-UIT/Cranfield/experimental_result.txt'
with open(res_path, 'w') as file:
    for idx in range(len(precision_lst)):
        string = str(idx+1) + ' ' + str(precision_lst[idx]) + ' ' + str(recall_lst[idx]) + ' ' + str(f_score_lst[idx]) + '\n'
        file.write(string)

In [ ]:
print('Độ chính xác trung bình:', round(sum(precision_lst)/len(precision_lst), 2))
print('Độ phủ trung bình:', round(sum(recall_lst)/len(recall_lst), 2))
print('Độ đo F trung bình:', round(sum(f_score_lst)/len(f_score_lst), 2))

Độ chính xác trung bình: 0.01
Độ phủ trung bình: 0.96
Độ đo F trung bình: 0.02


In [ ]:
!pip install rank_bm25

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi

In [ ]:
# Sử dụng hàm text_process để làm sạch dữ liệu
processed_docs = [" ".join(text_process(doc)) for doc in documents]
tokenized_docs = [text_process(doc) for doc in documents]

#VECTOR SPACE MODEL (TF-IDF)

In [ ]:
# Sử dụng hàm text_process để làm sạch dữ liệu
processed_docs = [" ".join(text_process(doc)) for doc in documents]
tokenized_docs = [text_process(doc) for doc in documents]

# --- VECTOR SPACE MODEL (TF-IDF) ---
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_docs)

def vsm_retrieval(query, top_k=10):
    query_vec = vectorizer.transform([" ".join(text_process(query))])
    sims = cosine_similarity(query_vec, tfidf_matrix).flatten()
    # Lấy index của top_k tài liệu có điểm cao nhất
    indices = sims.argsort()[-top_k:][::-1]
    return [idx + 1 for idx in indices] # Trả về list DocID (1-based)

# --- BM25 MODEL ---
bm25 = BM25Okapi(tokenized_docs)

def bm25_retrieval(query, top_k=10):
    tokenized_query = text_process(query)
    doc_scores = bm25.get_scores(tokenized_query)
    indices = doc_scores.argsort()[-top_k:][::-1]
    return [idx + 1 for idx in indices] # Trả về list DocID (1-based)

#MÔ HÌNH BM25

In [ ]:
def evaluate_model(retrieval_func, queries, relevance, is_ranked=True, top_k=10):
    precision_lst = []
    recall_lst = []
    f_score_lst = []

    for idx in range(len(queries)):
        # Lấy kết quả trả về từ mô hình
        if is_ranked:
            retrieved = set(retrieval_func(queries[idx], top_k=top_k))
        else:
            retrieved = set(retrieval_func(queries[idx], inverted_index)) # Cho Boolean

        relevant = set(relevance[idx + 1])
        common = retrieved & relevant

        p = len(common) / len(retrieved) if len(retrieved) > 0 else 0
        r = len(common) / len(relevant) if len(relevant) > 0 else 0
        f = (2 * p * r) / (p + r) if (p + r) > 0 else 0

        precision_lst.append(p)
        recall_lst.append(r)
        f_score_lst.append(f)

    avg_p = sum(precision_lst) / len(precision_lst)
    avg_r = sum(recall_lst) / len(recall_lst)
    avg_f = sum(f_score_lst) / len(f_score_lst)

    return round(avg_p, 4), round(avg_r, 4), round(avg_f, 4)

# Tính toán cho 3 mô hình (Lấy Top 10 cho các mô hình xếp hạng)
metrics_boolean = evaluate_model(boolean_retrieval, queries, relevance, is_ranked=False)
metrics_vsm = evaluate_model(vsm_retrieval, queries, relevance, is_ranked=True, top_k=1000)
metrics_bm25 = evaluate_model(bm25_retrieval, queries, relevance, is_ranked=True, top_k=1000)

# In kết quả
print(f"Boolean: Precision={metrics_boolean[0]}, Recall={metrics_boolean[1]}, F1={metrics_boolean[2]}")
print(f"VSM:     Precision={metrics_vsm[0]}, Recall={metrics_vsm[1]}, F1={metrics_vsm[2]}")
print(f"BM25:    Precision={metrics_bm25[0]}, Recall={metrics_bm25[1]}, F1={metrics_bm25[2]}")

# Tạo DataFrame so sánh
import pandas as pd
data = {
    "Mô hình": ["Boolean Retrieval", "Vector Space Model (Top 10)", "BM25 (Top 10)"],
    "Độ chính xác (P)": [metrics_boolean[0], metrics_vsm[0], metrics_bm25[0]],
    "Độ phủ (R)": [metrics_boolean[1], metrics_vsm[1], metrics_bm25[1]],
    "Độ đo F (F1)": [metrics_boolean[2], metrics_vsm[2], metrics_bm25[2]]
}
df_compare = pd.DataFrame(data)
display(df_compare)

Boolean: Precision=0.0096, Recall=0.9621, F1=0.0189
VSM:     Precision=0.0079, Recall=0.9695, F1=0.0156
BM25:    Precision=0.0079, Recall=0.9713, F1=0.0156


,Mô hình,Độ chính xác (P),Độ phủ (R),Độ đo F (F1)
0,Boolean Retrieval,0.0096,0.9621,0.0189
1,Vector Space Model (Top 10),0.0079,0.9695,0.0156
2,BM25 (Top 10),0.0079,0.9713,0.0156
